https://github.com/opencv/opencv/tree/master/data/haarcascades

https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html


https://docs.opencv.org/2.4/doc/user_guide/ug_traincascade.html

In [ ]:
import numpy as np
import cv2 as cv

rostro = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')

In [ ]:


cap = cv.VideoCapture(0)
x=y=w=h= 0 
img = 0
count = 0
while True:
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rostros = rostro.detectMultiScale(gray, 1.3, 5)
    for(x, y, w, h) in rostros:
        m= int(h/2)
        frame = cv.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)
       # frame = cv.rectangle(frame, (x,y+m), (x+w, y+h), (255, 0 ,0), 2 )
        frame2 = frame[y:y+h, x:x+w]
        cv.imshow('rostros2', frame2)
        cv.imwrite('./capturas/johann/'+str(i)+'.png', frame2)
       # img = 180- frame[y:y+h,x:x+w]
       # count = count + 1   

        # !  para que las imágenes tengan el mismo tamaño se debe usar la función resize 100x100 px
    #name = '/home/likcos/imgs/cara'+str(count)+'.jpg'
    #cv.imwrite(name, frame)
    cv.imshow('rostros', frame)
    #cv.imshow('cara', img)
    i = i+1 
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()
    

In [ ]:
pwd

# Codigo para realizar la capturas

In [ ]:
import cv2 as cv
import os

# Verificar si la carpeta de destino existe, si no, crearla
output_dir = './capturas/johann/triste/'
if not os.path.exists(output_dir):
    print('No existe el directorio')
    

# Cargar el clasificador de Haar para la detección de rostros
rostro = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')

# Iniciar la captura de vídeo desde la cámara
cap = cv.VideoCapture(0)

# Contador para el nombre de los archivos de salida
i = 333

while True:
    # Capturar un fotograma de la cámara
    ret, frame = cap.read()

    # Convertir el fotograma a escala de grises
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # Detectar rostros en la imagen en escala de grises
    rostros = rostro.detectMultiScale(gray, 1.3, 5)
    
    # Iterar sobre cada rostro detectado
    for(x, y, w, h) in rostros:
        # Dibujar un rectángulo alrededor del rostro detectado en el fotograma original
        
        # Dibujar un rectángulo alrededor del rostro detectado en el fotograma original
        frame = cv.rectangle(frame, (x-20, y-20), (x+w+20, y+h+20), (0, 255, 0), 2)

        # frame = cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Recortar el área del rostro detectado
        frame2 = frame[y:y+h, x:x+w]
        
        # Redimensionar el área del rostro a 100x100 píxeles
        frame2 = cv.resize(frame2, (100, 100), interpolation=cv.INTER_AREA)
        
        # Guardar el rostro redimensionado como una imagen PNG
        cv.imwrite(output_dir+'johanntriste'+str(i)+'.png', frame2)
        
    # Mostrar el fotograma con los rostros detectados
    cv.imshow('Rostros', frame)
    
    # Incrementar el contador de archivos de salida
    i += 1
    
    # Esperar por la tecla ESC para salir del bucle
    k = cv.waitKey(1)
    if k == 27:
        break

# Liberar los recursos de la cámara y cerrar las ventanas
cap.release()
cv.destroyAllWindows()

# Codigo para generar el xml

In [ ]:
import cv2 as cv 
import numpy as np 
import os

dataSet = './capturas'
emotions = ['triste', 'feliz', 'enojado', 'asombrado', 'neutro']
faces  = os.listdir(dataSet)

labels = []
facesData = []
label = 0 


# for face in faces:
#     facePath = dataSet+'/'+face
#     for faceName in os.listdir(facePath):
#         labels.append(label)
#         facesData.append(cv.imread(facePath+'/'+faceName,0))
#     label = label + 1
# print(np.count_nonzero(np.array(labels)==0)) 

# Iterar sobre cada emoción
for emotion in emotions:
    emotionPath = os.path.join(dataSet, emotion)
    if not os.path.isdir(emotionPath):
        continue  # Si el directorio de la emoción no existe, saltar

    # Iterar sobre cada persona dentro de la emoción
    for person in os.listdir(emotionPath):
        personPath = os.path.join(emotionPath, person)
        if not os.path.isdir(personPath):
            continue  # Si no es un directorio, saltar

        # Iterar sobre cada imagen de la persona
        for faceName in os.listdir(personPath):
            facePath = os.path.join(personPath, faceName)
            if os.path.isfile(facePath):
                labels.append(label)
                facesData.append(cv.imread(facePath, 0))
        label += 1

print(f"Total de imágenes de la emoción 'triste': {np.count_nonzero(np.array(labels) == 0)}")
print(f"Total de imágenes de la emoción 'feliz': {np.count_nonzero(np.array(labels) == 1)}")
print(f"Total de imágenes de la emoción 'enojado': {np.count_nonzero(np.array(labels) == 2)}")
print(f"Total de imágenes de la emoción 'asombrado': {np.count_nonzero(np.array(labels) == 3)}")
print(f"Total de imágenes de la emoción 'neutro': {np.count_nonzero(np.array(labels) == 4)}")

# faceRecognizer = cv.face.EigenFaceRecognizer_create()
# faceRecognizer.train(facesData, np.array(labels))
# faceRecognizer.write('laloEigenface.xml')

# Entrenar el reconocedor de caras
faceRecognizer = cv.face.EigenFaceRecognizer_create()
faceRecognizer.train(facesData, np.array(labels))
faceRecognizer.write('datasetEmotionEigenface.xml')

In [ ]:
import cv2
import os
import numpy as np

dataPath = './capturas' 
emotionList = os.listdir(dataPath)
print('Lista de emociones: ', emotionList)

labels = []
facesData = []
label = 0

# for nameDir in emotionList:
# 	personPath = dataPath + '/' + nameDir
# 	# print('Leyendo las imágenes')

# 	for fileName in os.listdir(personPath):
# 		# print('Rostros: ', nameDir + '/' + fileName)
# 		labels.append(label)
# 		facesData.append(cv2.imread(personPath+'/'+fileName,0))
# 		#image = cv2.imread(personPath+'/'+fileName,0)
# 		#cv2.imshow('image',image)
# 		#cv2.waitKey(10)
# 	label = label + 1

for emotionDir in emotionList:
    emotionPath = os.path.join(dataPath, emotionDir)
    print(f'Leyo las imágenes de la emoción: {emotionDir}')


    for personDir in os.listdir(emotionPath):
        personPath = os.path.join(emotionPath, personDir)

        for fileName in os.listdir(personPath):
            filePath = os.path.join(personPath, fileName)
    
            # Asegurarse de que el archivo es una imagen
            if os.path.isfile(filePath):
                image = cv2.imread(filePath, 0)
                if image is not None:
                    # Asegurarse de que todas las imágenes tienen el mismo tamaño
                    if facesData and image.shape != facesData[0].shape:
                        print(f'Error: las imágenes no tienen el mismo tamaño: {filePath}')
                        continue
                    labels.append(label)
                    facesData.append(image)
                    # cv2.imshow('image', image)
                    # cv2.waitKey(10)
                else:
                    print(f'Error al leer la imagen: {filePath}')
        label += 1


print('labels= ',labels)
print('Número de etiquetas 0: ',np.count_nonzero(np.array(labels)==0))
print('Número de etiquetas 1: ',np.count_nonzero(np.array(labels)==1))

# Métodos para entrenar el reconocedor
#face_recognizer = cv2.face.EigenFaceRecognizer_create()
#face_recognizer = cv2.face.FisherFaceRecognizer_create()
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

# Entrenando el reconocedor de rostros
print("Entrenando...")
face_recognizer.train(facesData, np.array(labels))

# Almacenando el modelo obtenido
#face_recognizer.write('modeloEigenFace.xml')
#face_recognizer.write('modeloFisherFace.xml')
face_recognizer.write('modeloLBPHFace.xml')
print("Modelo almacenado...")

# Codigo para cargar el xml

In [ ]:
import cv2 as cv
import os

cascade_path = 'haarcascade_frontalface_alt.xml'

# Inicializar el reconocedor de caras
faceRecognizer = cv.face.LBPHFaceRecognizer_create()

# Leer el modelo entrenado
model_path = "EmocionesmodeloLBPHFace.xml"
if os.path.exists(model_path):
    print("El archivo del modelo existe")
    try:
        faceRecognizer.read(model_path)
        print("Modelo cargado con éxito")
    except Exception as e:
        print(f"Error al cargar el modelo: {e}")
else:
    raise Exception("El archivo del modelo no existe")

# Definir etiquetas de emociones
emotions = ["enojo", "feliz", "asombrado", "triste", "neutro"]

# Inicializar la captura de video
cap = cv.VideoCapture(0)
if not cap.isOpened():
    raise Exception("No se pudo abrir el dispositivo de video")

# Cargar el Haar cascade para detección de rostros
rostro = cv.CascadeClassifier(cascade_path)
if rostro.empty():
    raise Exception("Error al cargar el archivo Haar cascade")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error al capturar el cuadro")
        break
    
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()
    rostros = rostro.detectMultiScale(gray, 1.3, 3)
    
    for (x, y, w, h) in rostros:
        frame2 = cpGray[y:y+h, x:x+w]
        frame2 = cv.resize(frame2, (100, 100), interpolation=cv.INTER_CUBIC)
        result = faceRecognizer.predict(frame2)
        cv.putText(frame, f'{result}', (x, y-5), 1, 1.3, (255,255,0), 1, cv.LINE_AA)
        
        if result[1] < 70:
            cv.putText(frame, f'{emotions[result[0]]}', (x, y-25), 2, 1.1, (0, 255, 0), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        else:
            cv.putText(frame, 'No identificado', (x, y-20), 2, 0.8, (0, 0, 255), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
    
    cv.imshow('frame', frame)
    if cv.waitKey(1) == 27:  # Presiona 'ESC' para salir
        break

cap.release()
cv.destroyAllWindows()

In [1]:
import cv2 as cv
import os

# Verifica que exista el archivo
cascade_path = 'haarcascade_frontalface_alt.xml'
if not os.path.isfile(cascade_path):
    raise FileNotFoundError(f"El archivo '{cascade_path}' no existe")
else:
    print('si existe el archivo ', {cascade_path})

# Verifica que exista el archivo
model_path = "EmocionesLBPHZwei.xml"
if not os.path.isfile(model_path):
    raise FileNotFoundError(f"El archivo '{model_path}' no exite")
else:
    print('si existe el archivo ', {model_path})

# Initialize face recognizer
faceRecognizer = cv.face.LBPHFaceRecognizer_create()

# Read the trained model
faceRecognizer.read(model_path)

# Define emotion labels
faces = ["enojo", "feliz", "sorpresa"]

# Initialize video capture
cap = cv.VideoCapture(0)
if not cap.isOpened():
    raise Exception("Could not open video device")

# Load the Haar cascade for face detection
rostro = cv.CascadeClassifier(cascade_path)
if rostro.empty():
    raise Exception("Failed to load Haar cascade file")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break
    
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()
    rostros = rostro.detectMultiScale(gray, 1.3, 3)
    
    for (x, y, w, h) in rostros:
        frame2 = cpGray[y:y+h, x:x+w]
        frame2 = cv.resize(frame2, (100, 100), interpolation=cv.INTER_CUBIC)
        result = faceRecognizer.predict(frame2)
        cv.putText(frame, f'{result}', (x, y-5), 1, 1.3, (255,255,0), 1, cv.LINE_AA)
        
        # Display the prediction result
        # if result[1] < 100:
        #     cv.putText(frame, f'{faces[result[0]]}', (x, y-25), 2, 1.1, (0, 255, 0), 1, cv.LINE_AA)
        #     cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        # else:
        #     cv.putText(frame, 'Desconocido', (x, y-20), 2, 0.8, (0, 0, 255), 1, cv.LINE_AA)
        #     cv.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        if result[1] < 70:
            cv.putText(frame, f'{emotions[result[0]]}', (x, y-25), 2, 1.1, (0, 255, 0), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        else:
            cv.putText(frame, 'No identificado', (x, y-20), 2, 0.8, (0, 0, 255), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
    
    cv.imshow('frame', frame)
    if cv.waitKey(1) == 27:  # Press 'ESC' to exit
        break

cap.release()
cv.destroyAllWindows()

si existe el archivo  {'haarcascade_frontalface_alt.xml'}
si existe el archivo  {'EmocionesmodeloLBPHFace.xml'}


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\persistence.cpp:1601: error: (-215:Assertion failed) ofs == fs_data_blksz[blockIdx] in function 'cv::FileStorage::Impl::normalizeNodeOfs'
